In [90]:
import pandas as pd
import numpy as np
import os
import requests
import bs4
import imdb
from imdb import Cinemagoer
import tqdm
import collections
import json

In [91]:
movie_script_index_file = os.path.join(os.getenv("DATA_DIR"), 
                                       "narrative_understanding/movie_scripts/movie_scripts/index.csv")
movie_script_index_df = pd.read_csv(movie_script_index_file, index_col=None)

In [92]:
display(movie_script_index_df.head())
print("number of nan imdb ids:", movie_script_index_df["imdb_id"].isna().sum())
print("number of rows:", movie_script_index_df.shape[0])
print("number of non-nan unique imdb ids:", movie_script_index_df["imdb_id"].dropna().unique().size)

,url,file,date,imdb_id,moviedb_id,script_on_screen_synopsis
0,https://thescriptsavant.com/movies/Aaranya_Kaa...,0,Feb 15 2023,tt1496729,119123.0,A drama that unfolds between two rival mafia g...
1,https://thescriptsavant.com/movies/Abduction.pdf,1,Feb 15 2023,tt1600195,59965.0,A young man sets out to uncover the truth abou...
2,http://www.dailyscript.com/scripts/Above_the_L...,2,Feb 15 2023,tt0094602,9395.0,"Nico Toscani is an Italian immigrant, American..."
3,http://www.imsdb.com/scripts/Above-the-Law.html,3,Feb 15 2023,tt0094602,9395.0,"Nico Toscani is an Italian immigrant, American..."
4,http://www.dailyscript.com/scripts/Absolute_Po...,4,Feb 15 2023,tt0118548,66.0,A master thief coincidentally is robbing a hou...


number of nan imdb ids: 7
number of rows: 5928
number of non-nan unique imdb ids: 2662


In [5]:
ia = Cinemagoer()

In [6]:
movie = ia.get_movie("0133093")

In [17]:
movie.get_titlesRefs()

{'Back': <Movie id:0133093[http] title:_Back (None)_>,
 'Cast & crew': <Movie id:0133093[http] title:_Cast & crew (None)_>,
 'User reviews': <Movie id:0133093[http] title:_User reviews (None)_>,
 'Trivia': <Movie id:0133093[http] title:_Trivia (None)_>,
 'The Matrix (1999)': <Movie id:0133093[http] title:_The Matrix (1999)_>,
 'The Matrix Reloaded (2003)': <Movie id:0234215[http] title:_The Matrix Reloaded (2003)_>}

In [18]:
type(movie.get_titlesRefs())

dict

In [19]:
movie.get("title")

'The Matrix'

In [20]:
movie.get("year")

1999

In [21]:
movie.get("rubbish")

In [28]:
# print("all imdb ids start with 'tt'", movie_script_index_df["imdb_id"].dropna().str.match("tt").all())
print((movie_script_index_df["imdb_id"].dropna().str.slice(2).str.len() == 7).all())

False


In [33]:
collections.Counter(movie_script_index_df["imdb_id"].dropna().str.slice(2).str.len())

Counter({7: 5542, 8: 119})

In [29]:
movie_script_index_df[movie_script_index_df["imdb_id"].notna() 
                      & (movie_script_index_df["imdb_id"].str.slice(2).str.len() != 7)]

,url,file,date,imdb_id,moviedb_id,script_on_screen_synopsis
32,https://deadline.com/wp-content/uploads/2023/0...,32,Feb 15 2023,tt19770238,965150.0,Sophie reflects on the shared joy and private ...
146,https://deadline.com/wp-content/uploads/2022/1...,146,Feb 15 2023,tt10304142,664469.0,"In the 1930s, three friends—a doctor, a nurse,..."
170,https://lionsgate.brightspotcdn.com/54/d1/0f4d...,170,Feb 15 2023,tt10065694,627290.0,Successful author Veronica finds herself trapp...
200,https://amazonstudiosguilds.com/app/uploads/20...,200,Feb 15 2023,tt15301048,714888.0,"In the 1980s, a team of lawyers takes on the h..."
201,https://amazonstudiosguilds.com/app/uploads/20...,201,Feb 15 2023,tt15301048,714888.0,"In the 1980s, a team of lawyers takes on the h..."
...,...,...,...,...,...,...
5250,https://www.scriptslug.com/script/the-whale-2022,5250,Feb 16 2023,tt13833688,785084.0,A reclusive English teacher suffering from sev...
5499,https://d1qf1lxchdoll2.cloudfront.net/2022/uar...,5499,Feb 16 2023,tt13669038,777245.0,A group of women in an isolated Mennonite comm...
5500,https://www.scriptslug.com/script/women-talkin...,5500,Feb 16 2023,tt13669038,777245.0,A group of women in an isolated Mennonite comm...
5508,https://www.scriptslug.com/script/work-it-2020,5508,Feb 16 2023,tt10276470,612706.0,A brilliant but clumsy high school senior vows...


In [34]:
imdb_id_to_movie = {}
error_imdb_ids = []

for imdb_id in tqdm.tqdm(movie_script_index_df["imdb_id"].dropna().str.slice(2).unique()):
    try:
        movie = ia.get_movie(imdb_id)
        imdb_id_to_movie[imdb_id] = movie
    except Exception:
        error_imdb_ids.append(imdb_id)

 18%|█▊        | 468/2644 [23:40<2:10:05,  3.59s/it]2023-04-17 11:17:26,757 CRITICAL [imdbpy] /proj/sbaruah/anaconda3/envs/story/lib/python3.10/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt1134807/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: ''>},); kwds: {}
Traceback (most recent call last):
  File "/proj/sbaruah/anaconda3/envs/story/lib/python3.10/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/proj/sbaruah/anaconda3/envs/story/lib/python3.10/urllib/request.py", line 525, in open
    response = meth(req, response)
  File "/proj/sbaruah/anaconda3/envs/story/lib/python3.10/urllib/request.py", line 634, in http_response
    response = self.parent.error(
  File "/proj/sbaruah/anaconda3/envs/story/lib/python3.10/urllib/request.py", line 563, in error
    return self

In [35]:
print(len(imdb_id_to_movie), len(error_imdb_ids))

2643 1


In [89]:
class movieEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, imdb.Person.Person):
            try:
                return {"personID": o.personID, "name": o.data.get("name")}
            except Exception:
                print(type(o.currentRole))
        elif isinstance(o, imdb.Company.Company):
            return {"companyID": o.companyID, "name": o.data.get("name")}
        elif isinstance(o, imdb.Movie.Movie):
            return {"movieID": o.movieID, "name": o.data.get("name")}
        return super().default(o)

imdb_id_to_movie_data = {}
for imdb_id, movie in imdb_id_to_movie.items():
    imdb_id_to_movie_data[imdb_id] = movie.data
movie_data_file = os.path.join(os.getenv("DATA_DIR"), 
                               "narrative_understanding/movie_scripts/movie_scripts/imdb_id_to_movie.json")
json.dump(imdb_id_to_movie_data, open(movie_data_file, "w"), indent=2, cls=movieEncoder)